In [8]:
from pathlib import Path
import sys

import pandas as pd
from xgboost import XGBClassifier

rank = 0 
#sys.path[0] = f'{Path().resolve().parents[rank]}' # mymoduleが上の階層にある場合rankを調整してコメント解除
from mymodule import PipeLine, grid_search_cv
from kayano import age_categolize, cholesterol_mean

In [19]:
###################### グリッドサーチ用のパイプラインを用意 ########################
def pipe_1(df, test_size, to_grid_search):
    print(' '*15, 'ベースライン関数')
    pipe = PipeLine()
    pipe(df)
    # 以下グリッドサーチ
    pack = pipe.fold_out_split(test_size)
    gs_model = grid_search_cv(pack, **to_grid_search)
    return gs_model

def pipe_2(df, test_size, to_grid_search):
    print(' '*15, '年齢カテゴライズ')
    pipe = PipeLine()
    pipe(df)
    pipe.df_num = age_categolize(pipe.df_num)
    # 以下グリッドサーチ
    pack = pipe.fold_out_split(test_size)
    gs_model = grid_search_cv(pack, **to_grid_search)
    return gs_model

def pipe_3(df, test_size, to_grid_search):
    print(' '*15, 'コレステロール平均補完')
    df_cho = cholesterol_mean(df)  # HeartDiseaseを参照するため前もって処理
    pipe = PipeLine()
    pipe(df_cho)  # df_choを代入
    # 以下グリッドサーチ
    pack = pipe.fold_out_split(test_size)
    gs_model = grid_search_cv(pack, **to_grid_search)
    return gs_model

def pipe_4(df, test_size, to_grid_search):
    print(' '*15, '年齢補完 and コレステロール平均補完')
    df_cho = cholesterol_mean(df)  # HeartDiseaseを参照するため前もって処理
    pipe = PipeLine()
    pipe(df_cho)  # df_choを代入
    pipe.df_num = age_categolize(pipe.df_num)
    # 以下グリッドサーチ
    pack = pipe.fold_out_split(test_size)
    gs_model = grid_search_cv(pack, **to_grid_search)
    return gs_model

In [20]:
df = pd.read_csv('../data/train.csv')
#################### グリッドサーチの探索候補を用意 #################
tree_grid = {'max_depth':[5,  7, 9, 15],
              'learning_rate': [0.1, 0.3],
              'n_estimators': [50, 100],
            }

############################# 用意した関数に渡す引数 ##############################
to_grid_search = {'model': XGBClassifier,
                  'param_grid': tree_grid,
                  'model_arg': {'random_state': 42, 'early_stopping_rounds': 10}
                  }

pipe_kwargs = {'df': df,
               'test_size': 0.2,
               'to_grid_search': to_grid_search}

######################## 用意したパイプラインをfor文で訓練 #########################
pipe_lines = [pipe_1, pipe_2, pipe_3, pipe_4]  # 作ったdemo関数をリストでまとめる （）いらない
models = []  # 訓練したモデルの格納先
for pipe in pipe_lines:
    print('#'*20,f'{pipe.__name__}の結果', '#'*20)  # 表示結果のラベル
    model = pipe(**pipe_kwargs)
    models.append(model)
    print()


#################### pipe_1の結果 ####################
                ベースライン関数
-------------------- 評価結果 --------------------


,accuracy_score,precision_score,recall_score,f1_score
train,0.980507,0.978495,0.985560,0.982014
test,0.813953,0.862500,0.841463,0.851852



#################### pipe_2の結果 ####################
                年齢カテゴライズ
-------------------- 評価結果 --------------------


,accuracy_score,precision_score,recall_score,f1_score
train,0.914230,0.933086,0.906137,0.919414
test,0.829457,0.916667,0.804878,0.857143



#################### pipe_3の結果 ####################
                コレステロール平均補完
-------------------- 評価結果 --------------------


,accuracy_score,precision_score,recall_score,f1_score
train,0.933723,0.961977,0.913357,0.937037
test,0.852713,0.898734,0.865854,0.881988



#################### pipe_4の結果 ####################
                年齢補完 and コレステロール平均補完
-------------------- 評価結果 --------------------


,accuracy_score,precision_score,recall_score,f1_score
train,0.964912,0.977860,0.956679,0.967153
test,0.868217,0.933333,0.853659,0.891720


In [22]:
models[3].best_params_

{'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 50}